<a href="https://colab.research.google.com/github/pea-sys/Kaggle_Public/blob/master/KaggleTemplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kaggle開発環境作成
前提条件：kaggle.jsonをGoogleドライブのルート直下に配置

一度KaggleAPIでデータセットダウンロード後にpickleファイルをgoogledriveにアップロード  

テーブルコンペしか経験していないので実質テーブルコンペ専用



In [0]:
!pip install kaggle

In [0]:
import google.colab
import googleapiclient.discovery
import googleapiclient.http
google.colab.auth.authenticate_user()
drive_service = googleapiclient.discovery.build('drive', 'v3')

In [0]:
def gdrive_dl(filename):
  upload_filename = filename

  file_list = drive_service.files().list(q="name='" + upload_filename + "'").execute().get('files')

  # ファイル ID を取得します。
  file_id = None
  for file in file_list:
    if file.get('name') == upload_filename:
      file_id = file.get('id')
      break

  if file_id is None:
    # ファイル ID を取得できなかった場合はエラーメッセージを出力します。
    print(upload_filename + ' が見つかりません.')
  else:
    # colab 環境へファイルをアップロードします。
    with open(upload_filename, 'wb') as f:
      request = drive_service.files().get_media(fileId=file_id)
      media = googleapiclient.http.MediaIoBaseDownload(f, request)

      done = False
      while not done:
        progress_status, done = media.next_chunk()
        print(100*progress_status.progress(), end="")
        print("%完了")

    print('GoogleドライブからColab環境へのファイル取り込みが完了しました.')
    
def gdrive_up(filename):

  file_metadata = {'name': filename,'mimeType': 'application/octet-stream'}
  media = googleapiclient.http.MediaFileUpload(filename,mimetype='application/octet-stream',resumable=True)
  created = drive_service.files().create(body=file_metadata,media_body=media, fields='id').execute()

In [0]:
gdrive_dl("kaggle.json")

In [0]:
# kaggle.jsonへの読み書き権限を与える
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions list --group 'entered'

In [0]:
!kaggle competitions download -c 'Datasetをダウンロードしたいコンペ名'

In [0]:
!df -h

In [0]:
!ls 

## Common Library Import

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## LINE通知
[アクセストークン取得](https://notify-bot.line.me/my/ )  
長時間かかる処理はLINEに終了通知

In [0]:
import requests

def send_line_notification(message):
    line_token = 'アクセストークン入力'  # 終わったら無効化する
    endpoint = 'https://notify-api.line.me/api/notify'
    message = "\n{}".format(message)
    payload = {'message': message}
    headers = {'Authorization': 'Bearer {}'.format(line_token)}
    requests.post(endpoint, data=payload, headers=headers)

## DataFrame Memory Reduce
pandasのDataFrameはintならnp.int64に、floatならnp.float64がデフォルト。データセットが大きくなってくると、DataFrameがメモリを圧迫して学習を思うように進められない。
データロード時の型指定では特徴追加等に対応しきれないので取り回しが効く関数を使用


In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### DatasetLoad
メモリー削減済みDataFrameをGoogleDriveに保存しておくことで
データ読み込みを高速化

In [0]:
# コンペ内容によって適宜書き換え
# 初回は失敗する
gdrive_dl("sample_submission.pickle")
gdrive_dl("train.pickle")
gdrive_dl("test.pickle")

In [0]:
%%time
import pickle

if not os.path.isfile('sample_submission.pickle'):
  sub = reduce_mem_usage(pd.read_csv('sample_submission.csv.zip'))
  sub.to_pickle('sample_submission.pickle')
  gdrive_up('sample_submission.pickle')
else:
  sub = pd.read_pickle('sample_submission.pickle')

if not os.path.isfile('train.pickle'):
  train_identity = pd.read_csv('train_identity.csv.zip')
  train_transaction = pd.read_csv('train_transaction.csv.zip')
  train = reduce_mem_usage(pd.merge(train_transaction, train_identity, on='TransactionID', how='left'))
  train.to_pickle('train.pickle')
  del train_identity,train_transaction
  gdrive_up('train.pickle')
else:
  train = pd.read_pickle('train.pickle') 
 
if not os.path.isfile('test.pickle'):
  test_identity = pd.read_csv('test_identity.csv.zip')
  test_transaction = pd.read_csv('test_transaction.csv.zip')
  test = reduce_mem_usage(pd.merge(test_transaction, test_identity, on='TransactionID', how='left'))
  test.to_pickle('test.pickle')
  del test_identity,test_transaction
  gdrive_up('test.pickle')
else:
  test = pd.read_pickle('test.pickle') 
  

## Data OverView

In [0]:
## Dataframeのdescribe + 欠損割合
def describe_add_isnull(df):
  mis_val = df.isnull().sum()
  mis_val_percent = 100 * mis_val / len(df)
  mis_val_table = pd.DataFrame(mis_val_percent)
  mis_val_table_ren_columns = mis_val_table.rename(columns={0:"Missing % of Total Values"})
  return pd.concat([df.describe(include='all'),mis_val_table_ren_columns.T])